In [1]:
import numpy as np
n_features = 'all'
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10'
endpt = 16
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
l2_final=1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'

In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,5258.5,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,5560.5,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,5381.5,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100056526386,0,5511.5,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
4,100061300991,0,5403.5,0,1,200.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4196,416771521620,0,5586.5,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4197,416783315386,0,5563.5,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4198,416796224310,0,5452.5,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,444,445,446,447,448,449,450,451,452,453
0,100016012504,0,5258.5,-1.120553,-0.959853,0.239613,-1.105713,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.774994
1,100023004268,0,5560.5,-1.120553,-0.959853,-0.356702,-0.746751,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.981136
2,100033323702,0,5381.5,-1.120553,-0.959853,1.670768,-0.100620,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,1.668274
3,100056526386,0,5511.5,-1.120553,1.041826,0.627217,-0.674959,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.461855
4,100061300991,0,5403.5,-1.120553,1.041826,0.657033,-0.962128,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.736711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,0.892416,-0.959853,1.402426,0.258341,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,0.774994
4196,416771521620,0,5586.5,-1.120553,1.041826,0.776296,1.622396,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.599283
4197,416783315386,0,5563.5,0.892416,-0.959853,-0.565412,0.330134,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,0.853663,-0.089138,-0.121811,2.767696
4198,416796224310,0,5452.5,0.892416,1.041826,-0.416334,0.114757,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,1.462133


In [9]:
data_full = data_full

In [10]:
# decide on number of dicrete times:
#halflife=13.*365.25

#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [11]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [12]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    # l2_final=0.0001

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)






FOLD 0:

Train C-index fold 1 :
0.9196248735452002
Test C-index fold 1 :
0.706877964639931

FOLD 1:

Train C-index fold 2 :
0.8513787468664844
Test C-index fold 2 :
0.7149079821409126

FOLD 2:

Train C-index fold 3 :
0.8928975850074665
Test C-index fold 3 :
0.6883379522133174

FOLD 3:

Train C-index fold 4 :
0.8616736359675238
Test C-index fold 4 :
0.7432094353109364

FOLD 4:

Train C-index fold 5 :
0.8715296611419636
Test C-index fold 5 :
0.689329246040435

FOLD 5:

Train C-index fold 6 :
0.8793264457752129
Test C-index fold 6 :
0.7267294790482824

FOLD 6:

Train C-index fold 7 :
0.8860672747563967
Test C-index fold 7 :
0.6899238854657485

FOLD 7:

Train C-index fold 8 :
0.8719784184087888
Test C-index fold 8 :
0.6735565862031188

FOLD 8:

Train C-index fold 9 :
0.8606232737505862
Test C-index fold 9 :
0.7486140176965158

FOLD 9:

Train C-index fold 10 :
0.9211781949016725
Test C-index fold 10 :
0.6975077548400899

FOLD 10:

Train C-index fold 11 :
0.9311954268341249
Test C-index fol